In [2]:
from huggingface_hub import login

login()

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, M2M100ForConditionalGeneration, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForSeq2SeqLM, NllbTokenizerFast
from tokenization_small100 import SMALL100Tokenizer
from peft import LoraModel

W1119 10:07:40.005000 22796 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [5]:
from peft import LoraConfig, get_peft_model

FAIRSEQ_LANGUAGE_CODES = ['ace_Arab', 'ace_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'afr_Latn', 'ajp_Arab', 'aka_Latn', 'amh_Ethi', 'apc_Arab', 'arb_Arab', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'asm_Beng', 'ast_Latn', 'awa_Deva', 'ayr_Latn', 'azb_Arab', 'azj_Latn', 'bak_Cyrl', 'bam_Latn', 'ban_Latn', 'bel_Cyrl', 'bem_Latn', 'ben_Beng', 'bho_Deva', 'bjn_Arab', 'bjn_Latn', 'bod_Tibt', 'bos_Latn', 'bug_Latn', 'bul_Cyrl', 'cat_Latn', 'ceb_Latn', 'ces_Latn', 'cjk_Latn', 'ckb_Arab', 'crh_Latn', 'cym_Latn', 'dan_Latn', 'deu_Latn', 'dik_Latn', 'dyu_Latn', 'dzo_Tibt', 'ell_Grek', 'eng_Latn', 'epo_Latn', 'est_Latn', 'eus_Latn', 'ewe_Latn', 'fao_Latn', 'pes_Arab', 'fij_Latn', 'fin_Latn', 'fon_Latn', 'fra_Latn', 'fur_Latn', 'fuv_Latn', 'gla_Latn', 'gle_Latn', 'glg_Latn', 'grn_Latn', 'guj_Gujr', 'hat_Latn', 'hau_Latn', 'heb_Hebr', 'hin_Deva', 'hne_Deva', 'hrv_Latn', 'hun_Latn', 'hye_Armn', 'ibo_Latn', 'ilo_Latn', 'ind_Latn', 'isl_Latn', 'ita_Latn', 'jav_Latn', 'jpn_Jpan', 'kab_Latn', 'kac_Latn', 'kam_Latn', 'kan_Knda', 'kas_Arab', 'kas_Deva', 'kat_Geor', 'knc_Arab', 'knc_Latn', 'kaz_Cyrl', 'kbp_Latn', 'kea_Latn', 'khm_Khmr', 'kik_Latn', 'kin_Latn', 'kir_Cyrl', 'kmb_Latn', 'kon_Latn', 'kor_Hang', 'kmr_Latn', 'lao_Laoo', 'lvs_Latn', 'lij_Latn', 'lim_Latn', 'lin_Latn', 'lit_Latn', 'lmo_Latn', 'ltg_Latn', 'ltz_Latn', 'lua_Latn', 'lug_Latn', 'luo_Latn', 'lus_Latn', 'mag_Deva', 'mai_Deva', 'mal_Mlym', 'mar_Deva', 'min_Latn', 'mkd_Cyrl', 'plt_Latn', 'mlt_Latn', 'mni_Beng', 'khk_Cyrl', 'mos_Latn', 'mri_Latn', 'zsm_Latn', 'mya_Mymr', 'nld_Latn', 'nno_Latn', 'nob_Latn', 'npi_Deva', 'nso_Latn', 'nus_Latn', 'nya_Latn', 'oci_Latn', 'gaz_Latn', 'ory_Orya', 'pag_Latn', 'pan_Guru', 'pap_Latn', 'pol_Latn', 'por_Latn', 'prs_Arab', 'pbt_Arab', 'quy_Latn', 'ron_Latn', 'run_Latn', 'rus_Cyrl', 'sag_Latn', 'san_Deva', 'sat_Beng', 'scn_Latn', 'shn_Mymr', 'sin_Sinh', 'slk_Latn', 'slv_Latn', 'smo_Latn', 'sna_Latn', 'snd_Arab', 'som_Latn', 'sot_Latn', 'spa_Latn', 'als_Latn', 'srd_Latn', 'srp_Cyrl', 'ssw_Latn', 'sun_Latn', 'swe_Latn', 'swh_Latn', 'szl_Latn', 'tam_Taml', 'tat_Cyrl', 'tel_Telu', 'tgk_Cyrl', 'tgl_Latn', 'tha_Thai', 'tir_Ethi', 'taq_Latn', 'taq_Tfng', 'tpi_Latn', 'tsn_Latn', 'tso_Latn', 'tuk_Latn', 'tum_Latn', 'tur_Latn', 'twi_Latn', 'tzm_Tfng', 'uig_Arab', 'ukr_Cyrl', 'umb_Latn', 'urd_Arab', 'uzn_Latn', 'vec_Latn', 'vie_Latn', 'war_Latn', 'wol_Latn', 'xho_Latn', 'ydd_Hebr', 'yor_Latn', 'yue_Hant', 'zho_Hans', 'zho_Hant', 'zul_Latn']  # fmt: skip

tokenizer = NllbTokenizerFast.from_pretrained("facebook/nllb-200-distilled-600M",
                                                src_lang="tgl_Latn",
                                                tgt_lang="bcl_Latn",)
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M", quantization_config=bnb_config)

tokenizer.add_special_tokens({"additional_special_tokens": ['bcl_Latn']}, replace_additional_special_tokens=False)
#tokenizer.pad_token = tokenizer.eos_token

1

In [6]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "v_proj"],
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)

peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 616,253,440 || trainable%: 0.1914


In [7]:
tokens = tokenizer("How is you day today?", return_tensors="pt").to(model.device)
translated_tokens = model.generate(
    **tokens, forced_bos_token_id=tokenizer.convert_tokens_to_ids("tgl_Latn"), max_length=30,
)
text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

In [8]:
text

['Kumusta ka ngayon?']

In [34]:
sentence_to_words = (lambda x: str(x)
                                 .replace(',', ' ')
                                 .replace('-', ' ')
                                 .replace('"', ' ')
                                 .replace('!', ' ')
                                 .replace(';', ' ')
                                 .replace('-', ' ')
                                 .replace('&', ' ')
                                 .replace('$', ' ')
                                 .replace('?', ' ')
                                 .replace('(', ' ')
                                 .replace(')', ' ')
                                 .replace('[', ' ')
                                 .replace(']', ' ')
                                 .replace('*', ' ')
                                 .replace('«', ' ')
                                 .replace('»', ' ')
                                 .replace('¡', ' ')
                                 .replace('¿', ' ')
                                 .replace('/', ' ')
                                 .replace('~', ' ')
                                 .replace('=', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('  ', ' ')
                                 .replace('\r\n', '\n')
                                 .replace('\n', ' ')
                                 .strip())

In [41]:
import pandas as pd
import datasets

parallel_corpora = pd.read_csv("tagalog-to-bicol-corpora.csv")

def preprocess(batch):
    model_inputs = tokenizer(
        batch["language1_text_words"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

    labels = tokenizer(
        batch["language2_text_words"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

corpora = parallel_corpora.sample(frac=1, random_state=42)

corpora['language1_text_words'] = corpora['language1_text'].apply(sentence_to_words)
corpora['language2_text_words'] = corpora['language2_text'].apply(sentence_to_words)

train_df = corpora.sample(frac=0.90, random_state=42)
eval_df = corpora.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

train_dataset = datasets.Dataset.from_pandas(train_df)
eval_dataset = datasets.Dataset.from_pandas(eval_df)

train_dataset_processed = train_dataset.map(preprocess, batched=True, remove_columns=['language1_text', 'language2_text', 'language1_text_words', 'language2_text_words'])
eval_dataset_processed = eval_dataset.map(preprocess, batched=True, remove_columns=['language1_text', 'language2_text', 'language1_text_words', 'language2_text_words'])

Map:   0%|          | 0/25367 [00:00<?, ? examples/s]

Map:   0%|          | 0/2819 [00:00<?, ? examples/s]

In [42]:
train_df

,Unnamed: 0,language1_text,language2_text,language1_text_words,language2_text_words
0,253191,"Kaya't lumayo sila sa tolda nina Kora, Datan a...","Kaya nagharayo sinda sa tolda ni Kora, ni Data...",Kaya't lumayo sila sa tolda nina Kora Datan at...,Kaya nagharayo sinda sa tolda ni Kora ni Datan...
1,260217,Nang ikatlong taon ng kanyang paghahari ay kan...,"Kan ikatolong taon nin saiyang paghahade, ipin...",Nang ikatlong taon ng kanyang paghahari ay kan...,Kan ikatolong taon nin saiyang paghahade ipina...
2,258630,"Nang dumating ang sugo at sinabi sa kanya, ""Ka...",Kan paisihon si Jehu na an mga payo kan mga ak...,Nang dumating ang sugo at sinabi sa kanya Kani...,Kan paisihon si Jehu na an mga payo kan mga ak...
3,269916,"Sa alinmang bayan o nayon kayo makapasok, hana...","""Pag-abot nindo sa sarong banwaan o baryo, han...",Sa alinmang bayan o nayon kayo makapasok hanap...,Pag abot nindo sa sarong banwaan o baryo hanap...
4,252864,"""Bibilangin mo rin ang bilang ng mga anak ni G...",na sensohon an Levitang kamaganakan ni Gerson ...,Bibilangin mo rin ang bilang ng mga anak ni Ge...,na sensohon an Levitang kamaganakan ni Gerson ...
...,...,...,...,...,...
25362,270598,"Subalit nangyari ang lahat ng ito, upang matup...",Alagad an gabos na ini nangyari nganing maotob...,Subalit nangyari ang lahat ng ito upang matupa...,Alagad an gabos na ini nangyari nganing maotob...
25363,276131,Huwag mong pabayaan ang kaloob na sa iyo'y ibi...,"Dai mo pagpabayaan an balaog na yaon saimo, na...",Huwag mong pabayaan ang kaloob na sa iyo'y ibi...,Dai mo pagpabayaan an balaog na yaon saimo na ...
25364,276650,"at kapag ang pagnanasang iyon ay naipaglihi, i...",Kun an mga maraot na kamawotan na iyan nakagam...,at kapag ang pagnanasang iyon ay naipaglihi it...,Kun an mga maraot na kamawotan na iyan nakagam...
25365,256115,Ang Panginoon ay pumapatay at bumubuhay; siya ...,An Kagurangnan an minagadan asin minabuhay liw...,Ang Panginoon ay pumapatay at bumubuhay siya a...,An Kagurangnan an minagadan asin minabuhay liw...


In [43]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="training-nllb-tgl-to-bicol-v2-working",
    learning_rate=5e-4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    logging_strategy="steps",
    logging_steps=500,
    logging_first_step=True,
    report_to="none",
    predict_with_generate=True,
)

In [44]:
from transformers import Seq2SeqTrainer
from peft import LoraConfig
import torch

import numpy as np

import evaluate

metric = evaluate.load("accuracy")
bleu = evaluate.load("bleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)

    if isinstance(preds, torch.Tensor):
        preds = preds.tolist()
    if isinstance(labels, torch.Tensor):
        labels = labels.tolist()
        
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = bleu.compute(predictions=decoded_preds,
                            references=[[l] for l in decoded_labels])
    return {"bleu": result["bleu"]}

trainer = Seq2SeqTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset_processed,
    eval_dataset=eval_dataset_processed,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\rek\AppData\Local\Temp\ipykernel_22796\1022429619.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [45]:
trainer.train()

Step,Training Loss,Validation Loss,Bleu
500,7.439400,7.277344,0.112899
1000,7.473400,7.285156,0.118221
1500,7.549000,7.339844,0.122536
2000,7.597800,7.425781,0.118900
2500,7.716500,7.562500,0.128968
3000,7.862500,7.703125,0.129005
3500,8.031700,7.894531,0.139360
4000,8.221700,8.085938,0.132315
4500,8.408200,8.242188,0.136766
5000,8.552100,8.351562,0.135123


TrainOutput(global_step=6342, training_loss=8.062293200340982, metrics={'train_runtime': 20693.2106, 'train_samples_per_second': 2.452, 'train_steps_per_second': 0.306, 'total_flos': 1.3789199075377152e+16, 'train_loss': 8.062293200340982, 'epoch': 2.0})

In [46]:
trainer.save_model("nllb-tgl-to-bicol-seq2seq-model-v2")    

In [47]:
tokens = tokenizer("Ano ang ginagawa mo?", return_tensors="pt").to(model.device)
translated_tokens = trainer.model.generate(
    **tokens, forced_bos_token_id=tokenizer.convert_tokens_to_ids("bcl_Latn"), max_length=30,
)
text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
text

['Ano anoman anoman anoman anoman anoman anoman anoman anoman anoman anoman anoman anoman anoman an']

In [48]:
generated_tokens = trainer.model.generate(**tokens)

In [49]:
generated_tokens

tensor([[     2, 256174,  15842,    111,   7481,   2735,    298,    316, 248130,
              2]], device='cuda:0')